# **Project: Football Data Analysis**

## Table of Contents

# **Introduction**

---

> This is an extensive dataset of football/soccer teams, players, leagues and matches played for the duration of 2008 to 2016. It is a dataset suited for machine learning but I am just performing exploratory data analysis with it. As a result, I will not be making use of a lot of the data from this dataset.
> 
> After studying the tables, I have come up with some questions that I plan to explore:
> 
> 1. 

# **Loading The Data**

---


### Importing all modules and packages that will be used


In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

import sqlite3
from sqlalchemy import create_engine

import pprint


UsageError: Line magic function `%` not found.


### Connecting to the SQLite database


In [47]:
# This code connects to the database through sqlite3
# This enables the database to be queried with traditional SQL code
load = sqlite3.connect('database.sqlite')
cursor = load.cursor()

# This code connects to the database through sqlalchemy
# This enables the database tables to be stored in a pandas dataframe and analyzed with python code and packages
engine = create_engine('sqlite:///database.sqlite')


### Listing all tables in the database


In [48]:
# This code checks the database to list all tables inside
cursor.execute('SELECT name from sqlite_master where type= "table"')
pprint.pprint(cursor.fetchall())


[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]


> Even though the above list contains 8 tables, there are actually 7 tables in the database. The first table `sqlite_sequence` is automatically generated by [SQLite](https://www.sqlite.org/autoinc.html#:~:text=The%20sqlite_sequence%20table%20is%20created,%2C%20INSERT%2C%20and%20DELETE%20statements.) to keep track of the other tables.


### Creating dataframes for each of the 7 tables


In [49]:
df_league = pd.read_sql_table("League", engine)
df_player = pd.read_sql_table("Player", engine)
df_match = pd.read_sql_table("Match", engine)
df_country = pd.read_sql_table("Country", engine)
df_team = pd.read_sql_table("Team", engine)
df_team_attr = pd.read_sql_table("Team_Attributes", engine)
df_player_attr = pd.read_sql_table("Player_Attributes", engine)

# cursor.close()


_From this point on, I try to run my analysis in both Python and SQL code._


# **Wrangling The Data**

---


## **1. `Match` Table**


In [50]:
df_match.head()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


### Removing Columns

> The `Match` table is very large and contains many columns which I would not need for my current analysis (especially the betting odds columns), some columns are missing all their values and some columns are not explained in the documentation so I don't know what they represent. I drop all irrelevant columns.
>
> I will be removing:
>
> 1. all columns that have **player** in their name, because I do not know what those numbers represent and it is not explained on the documentation
> 2. all the betting odds columns at the right side of the table, because I am not using them for my analysis
> 3. all the match statistics colummns (except the goals scored), because the values are not available in the dataset, they are all missing


In [88]:
# Listing all column names in the `match` table so I can know the index of the columns to be deleted
list(enumerate(df_match.columns, start=0))

[(0, 'id'),
 (1, 'country_id'),
 (2, 'league_id'),
 (3, 'season'),
 (4, 'stage'),
 (5, 'date'),
 (6, 'match_api_id'),
 (7, 'home_team_api_id'),
 (8, 'away_team_api_id'),
 (9, 'home_team_goal'),
 (10, 'away_team_goal'),
 (11, 'home_player_X1'),
 (12, 'home_player_X2'),
 (13, 'home_player_X3'),
 (14, 'home_player_X4'),
 (15, 'home_player_X5'),
 (16, 'home_player_X6'),
 (17, 'home_player_X7'),
 (18, 'home_player_X8'),
 (19, 'home_player_X9'),
 (20, 'home_player_X10'),
 (21, 'home_player_X11'),
 (22, 'away_player_X1'),
 (23, 'away_player_X2'),
 (24, 'away_player_X3'),
 (25, 'away_player_X4'),
 (26, 'away_player_X5'),
 (27, 'away_player_X6'),
 (28, 'away_player_X7'),
 (29, 'away_player_X8'),
 (30, 'away_player_X9'),
 (31, 'away_player_X10'),
 (32, 'away_player_X11'),
 (33, 'home_player_Y1'),
 (34, 'home_player_Y2'),
 (35, 'home_player_Y3'),
 (36, 'home_player_Y4'),
 (37, 'home_player_Y5'),
 (38, 'home_player_Y6'),
 (39, 'home_player_Y7'),
 (40, 'home_player_Y8'),
 (41, 'home_player_Y9'),
 (

In [82]:
# Storing the table columns in a variable
match_columns = df_match.columns

# Creating a list of all columns to be deleted
# match_columns_to_delete = [head for head in match_columns if head.find('player')>0 or head.find('365')>0 or len(head)==3]
match_columns_to_delete = match_columns[11:]

# Confirming changes
match_columns_to_delete


Index(['home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8',
       'home_player_X9', 'home_player_X10',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=104)

> I store the new table in a new dataframe instead of replacing the original one because I may descide to get some information from the original table in the future.

In [83]:
# Dropping unwanted columns
df_match_ed = df_match.drop(columns=match_columns_to_delete)

# Confirming changes
df_match_ed.head()


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3


### Check for duplicate rows

In [89]:
# Checking for duplicate rows
df_match_ed.duplicated().sum()

id                  0
country_id          0
league_id           0
season              0
stage               0
date                0
match_api_id        0
home_team_api_id    0
away_team_api_id    0
home_team_goal      0
away_team_goal      0
dtype: int64

### Check for null values

In [90]:
# Checking for null values
df_match_ed.isnull().sum()

id                  0
country_id          0
league_id           0
season              0
stage               0
date                0
match_api_id        0
home_team_api_id    0
away_team_api_id    0
home_team_goal      0
away_team_goal      0
dtype: int64

### Final check

In [109]:
df_match_ed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                25979 non-null  int64 
 1   country_id        25979 non-null  int64 
 2   league_id         25979 non-null  int64 
 3   season            25979 non-null  object
 4   stage             25979 non-null  int64 
 5   date              25979 non-null  object
 6   match_api_id      25979 non-null  int64 
 7   home_team_api_id  25979 non-null  int64 
 8   away_team_api_id  25979 non-null  int64 
 9   home_team_goal    25979 non-null  int64 
 10  away_team_goal    25979 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 2.2+ MB


## **2. `Team` Table**

In [94]:
df_team.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


### Check for duplicates

In [95]:
df_team.duplicated().sum()

0

### Check for null values

In [96]:
df_team.isnull().sum()

id                   0
team_api_id          0
team_fifa_api_id    11
team_long_name       0
team_short_name      0
dtype: int64

> There are 11 rows with missing data on the `team_fifa_api_id` column but I will not really be needing that column as since I can use the `team_api_id` as a foreign key to the other tables, so I will be dropping that column

In [100]:
# Dropping column
df_team_ed = df_team.drop(columns=['team_fifa_api_id'])

# Confirming changes
df_team_ed.head()

,id,team_api_id,team_long_name,team_short_name
0,1,9987,KRC Genk,GEN
1,2,9993,Beerschot AC,BAC
2,3,10000,SV Zulte-Waregem,ZUL
3,4,9994,Sporting Lokeren,LOK
4,5,9984,KSV Cercle Brugge,CEB


### Final check

In [110]:
df_team_ed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               299 non-null    int64 
 1   team_api_id      299 non-null    int64 
 2   team_long_name   299 non-null    object
 3   team_short_name  299 non-null    object
dtypes: int64(2), object(2)
memory usage: 9.5+ KB


## **3. `Player` Table**

In [113]:
df_player.head()
# df_player.info()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182,154


> I don't need the `player_fifa_api_id` column so I drop it

In [114]:
# Dropping column
df_player_ed = df_player.drop(columns='player_fifa_api_id')

# Confirming changes
df_player_ed

,id,player_api_id,player_name,birthday,height,weight
0,1,505942,Aaron Appindangoye,1992-02-29 00:00:00,182,187
1,2,155782,Aaron Cresswell,1989-12-15 00:00:00,170,146
2,3,162549,Aaron Doran,1991-05-13 00:00:00,170,163
3,4,30572,Aaron Galindo,1982-05-08 00:00:00,182,198
4,5,23780,Aaron Hughes,1979-11-08 00:00:00,182,154
...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,1979-04-03 00:00:00,182,168
11056,11072,111182,Zsolt Laczko,1986-12-18 00:00:00,182,176
11057,11073,36491,Zsolt Low,1979-04-29 00:00:00,180,154
11058,11074,35506,Zurab Khizanishvili,1981-10-06 00:00:00,185,172


# **Conclusions**

---

> The dataset is not very well explained in the documentation and there are many columns in come of the tables which I do not know what they represent so I drop them and try to reduce the tables to something I can work with.

# Misc queries

In [ ]:
# df_team_attr.sort_values(by='team_api_id')

# df_team.query('team_long_name == "Manchester City"')
# df_team[df_team['team_long_name'] == 'Manchester City']
# df_team[df_team.team_long_name == 'Manchester City']

# df_team.query('team_fifa_api_id.isnull()')

# cursor.execute(""
#                "SELECT * FROM Team WHERE team_fifa_api_id is Null"""
#                )
# pprint.pprint(cursor.fetchall())

# df_match_ed[df_match_ed.home_team_api_id == 8456]


In [ ]:
# SQLite query to filter the match to return all Manchester City matches played in 2015 

# cursor.execute(""
#                "SELECT * FROM Match WHERE home_team_api_id=8456 AND SUBSTR(date, 1, 4) = '2015' """
#                )
# pprint.pprint(cursor.fetchone())